### prepare results

In [34]:
import pandas as pd

from sklearn.metrics import mean_absolute_error as mae

In [18]:
train = pd.read_csv("../data/train.csv")
preds_oob = pd.read_csv("preds_oob.csv")
preds_test = pd.read_csv("submission-keras-shift-perm.csv")

In [20]:
preds_oob = preds_oob.sort_values(by="id")
preds_oob.columns = ["id","prediction"]
train_loss = train[["id", "loss"]]

In [25]:
preds_oob = preds_oob.merge(train_loss, on="id", how="inner")

In [28]:
preds_oob.to_csv("trainsubmission.csv", index=False)

In [32]:
preds_oob.head()

,id,prediction,loss
0,1,1782.613208,2213.18
1,2,1515.220886,1283.60
2,5,3978.569092,3005.09
3,10,943.790808,939.85
4,11,3238.237183,2763.85


In [35]:
print mae(preds_oob.loss.values, preds_oob.prediction.values)

1131.64728289


In [40]:
isotrain_200 = pd.read_csv("iso_reg/isotrain_200.csv")
print mae(isotrain_200.loss.values, isotrain_200.prediction.values)

1131.53175161


In [41]:
isotrain_50 = pd.read_csv("iso_reg/isotrain_50.csv")
print mae(isotrain_50.loss.values, isotrain_50.prediction.values)

1131.53175161


In [1]:
## import libraries
import numpy as np
np.random.seed(123)

import pickle
import pandas as pd
import subprocess
from scipy.sparse import csr_matrix, hstack
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import KFold
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU

/home/y.romanikhin/.virtualenvs/venv/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "/home/y.romanikhin/.virtualenvs/venv/lib/python2.7/site-packages/tensorflow/python/__init__.py", line 49, in <module>
    from tensorflow.python import pywrap_tensorflow
  File "/home/y.romanikhin/.virtualenvs/venv/lib/python2.7/site-packages/tensorflow/python/pywrap_tensorflow.py", line 28, in <module>
    _pywrap_tensorflow = swig_import_helper()
  File "/home/y.romanikhin/.virtualenvs/venv/lib/python2.7/site-packages/tensorflow/python/pywrap_tensorflow.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow', fp, pathname, description)
ImportError: /lib64/libc.so.6: version `GLIBC_2.16' not found (required by /home/y.romanikhin/.virtualenvs/venv/lib/python2.7/site-packages/tensorflow/python/_pywrap_tensorflow.so)


Error importing tensorflow.  Unless you are using bazel,
you should not try to import tensorflow from its source directory;
please exit the tensorflow source tree, and relaunch your python 

In [2]:
## Batch generators ##################################################################################################################################

def batch_generator(X, y, batch_size, shuffle):
    #chenglong code for fiting from generator (https://www.kaggle.com/c/talkingdata-mobile-user-demographics/forums/t/22567/neural-network-for-sparse-matrices)
    number_of_batches = np.ceil(X.shape[0]/batch_size)
    counter = 0
    sample_index = np.arange(X.shape[0])
    if shuffle:
        np.random.shuffle(sample_index)
    while True:
        batch_index = sample_index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X[batch_index,:].toarray()
        y_batch = y[batch_index]
        counter += 1
        yield X_batch, y_batch
        if (counter == number_of_batches):
            if shuffle:
                np.random.shuffle(sample_index)
            counter = 0

def batch_generatorp(X, batch_size, shuffle):
    number_of_batches = X.shape[0] / np.ceil(X.shape[0]/batch_size)
    counter = 0
    sample_index = np.arange(X.shape[0])
    while True:
        batch_index = sample_index[batch_size * counter:batch_size * (counter + 1)]
        X_batch = X[batch_index, :].toarray()
        counter += 1
        yield X_batch
        if (counter == number_of_batches):
            counter = 0

########################################################################################################################################################

In [3]:
# neural net
def nn_model():
    model = Sequential()
    
    model.add(Dense(400, input_dim = xtrain.shape[1], init = 'he_normal'))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
        
    model.add(Dense(200, init = 'he_normal'))
    model.add(PReLU())
    model.add(BatchNormalization())    
    model.add(Dropout(0.2))
    
    model.add(Dense(50, init = 'he_normal'))
    model.add(PReLU())
    model.add(BatchNormalization())    
    model.add(Dropout(0.2))
    
    model.add(Dense(1, init = 'he_normal'))
    model.compile(loss = 'mae', optimizer = 'adadelta')
    return(model)

In [4]:
## read data
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

index = list(train.index)
print index[0:10]
np.random.shuffle(index)
print index[0:10]
train = train.iloc[index]
'train = train.iloc[np.random.permutation(len(train))]'

## set test loss to NaN
test['loss'] = np.nan

## response and IDs
y = np.log(train['loss'].values+200)
id_train = train['id'].values
id_test = test['id'].values

## stack train test
ntrain = train.shape[0]
tr_te = pd.concat((train, test), axis = 0)

## Preprocessing and transforming to sparse data
sparse_data = []

f_cat = [f for f in tr_te.columns if 'cat' in f]
for f in f_cat:
    dummy = pd.get_dummies(tr_te[f].astype('category'))
    tmp = csr_matrix(dummy)
    sparse_data.append(tmp)

f_num = [f for f in tr_te.columns if 'cont' in f]
scaler = StandardScaler()
tmp = csr_matrix(scaler.fit_transform(tr_te[f_num]))
sparse_data.append(tmp)

del(tr_te, train, test)

## sparse train and test data
xtr_te = hstack(sparse_data, format = 'csr')
xtrain = xtr_te[:ntrain, :]
xtest = xtr_te[ntrain:, :]

print('Dim train', xtrain.shape)
print('Dim test', xtest.shape)

del(xtr_te, sparse_data, tmp)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[33898, 43407, 3458, 100139, 60086, 117290, 89806, 7751, 164059, 143297]
('Dim train', (188318, 1190))
('Dim test', (125546, 1190))


In [5]:
xtrain

<188318x1190 sparse matrix of type '<type 'numpy.float64'>'
	with 24481340 stored elements in Compressed Sparse Row format>

In [6]:
id_train.shape

(188318,)

In [7]:
## cv-folds
nfolds = 3
folds = KFold(len(y), n_folds = nfolds, shuffle = True, random_state = 111)

## train models
i = 0
nbags = 10
nepochs = 55
pred_oob = np.zeros(xtrain.shape[0])
pred_test = np.zeros(xtest.shape[0])

for (inTr, inTe) in folds:
    xtr = xtrain[inTr]
    ytr = y[inTr]
    xte = xtrain[inTe]
    yte = y[inTe]
    pred = np.zeros(xte.shape[0])

In [9]:
pred.shape

(62772,)

In [11]:
xte.shape

(62772, 1190)

In [19]:
train = pd.read_csv("train_big.csv")
test = pd.read_csv("test_big.csv")

In [20]:
shift = 200

ids = pd.read_csv('data/test.csv')['id']
y_train = np.log(train['loss'] + shift)
X_train = train.drop(['loss','id'], axis=1)
X_test = test.drop(['loss','id'], axis=1)

In [41]:
xy = csr_matrix(X_train)

MemoryError: 

In [34]:
nfolds = 10
folds = KFold(X_train.shape[0], n_folds = nfolds, shuffle = True, random_state = 111)

## train models
i = 0
nbags = 10
nepochs = 55
pred_oob = np.zeros(X_train.shape[0])
pred_test = np.zeros(X_test.shape[0])

for (inTr, inTe) in folds:
    xtr = X_train[inTr]
    ytr = y_train[inTr]
    xte = X_train[inTe]
    yte = y_train[inTe]
    pred = np.zeros(xte.shape[0])

IndexError: indices are out-of-bounds

In [ ]:

## cv-folds
nfolds = 10
folds = KFold(len(y_train), n_folds = nfolds, shuffle = True, random_state = 111)

## train models
i = 0
nbags = 10
nepochs = 55
pred_oob = np.zeros(X_train.shape[0])
pred_test = np.zeros(X_test.shape[0])

for (inTr, inTe) in folds:
    xtr = X_train[inTr]
    ytr = y_train[inTr]
    xte = X_train[inTe]
    yte = y_train[inTe]
    pred = np.zeros(xte.shape[0])
    for j in range(nbags):
        model = nn_model()
        fit = model.fit_generator(generator = batch_generator(xtr, ytr, 128, True),
                                  nb_epoch = nepochs,
                                  samples_per_epoch = xtr.shape[0],
                                  verbose = 0)
        pred += np.exp(model.predict_generator(generator = batch_generatorp(xte, 800, False), val_samples = xte.shape[0])[:,0])-shift
        pred_test += np.exp(model.predict_generator(generator = batch_generatorp(xtest, 800, False), val_samples = xtest.shape[0])[:,0])-shift
    pred /= nbags
    pred_oob[inTe] = pred
    score = mean_absolute_error(np.exp(yte)-200, pred)
    i += 1
    print('Fold ', i, '- MAE:', score)

print('Total - MAE:', mean_absolute_error(np.exp(y)-200, pred_oob))

with open('pred_oob.pickle', 'wb') as f:
    pickle.dump(pred_oob, f)

with open('pred_test.pickle', 'wb') as f:
    pickle.dump(pred_test, f)

## train predictions
df = pd.DataFrame({'id': train.id.values, 'loss': pred_oob})
df.to_csv('preds_oob.csv', index = False)

## test predictions
pred_test /= (nfolds*nbags)
df = pd.DataFrame({'id': test.id.values, 'loss': pred_test})
df.to_csv('submission_keras_shift_perm.csv', index = False)